In [ ]:
def draw_ols_result(ols_result):
    import numpy as np
    import matplotlib.pyplot as plt
    import scipy.stats as stats
    # 
    betas   = list(ols_result.params.index)
    dof     = ols_result.df_resid
    # 
    fig, axs = plt.subplots(nrows=len(betas), figsize=(10, 4*len(betas)))
    # 
    for ax, beta in zip(axs, betas) :
        b_mean  = ols_result.params[beta]
        b_se    = ols_result.bse[beta]
        b_conf  = [ols_result.conf_int().loc[beta][0], ols_result.conf_int().loc[beta][1]]
        p_val   = ols_result.pvalues[beta]
        # 
        x = np.linspace(min(0, b_mean - 4 * b_se), (b_mean + 4 * b_se), 1000)
        y = stats.t.pdf(x, df=dof, loc=b_mean, scale=b_se)
        # 
        ax.plot(x, y, label=f'$b_{beta}$={b_mean:.3f}, $s_{beta}^2$={b_se:.3f}$^2$))')
        # 
        # Plot the critical values
        ax.axvline(b_conf[1], linestyle='--', alpha=0.5, color='red')
        ax.axvline(b_conf[0], linestyle='--', alpha=0.5, color='green')              
        ax.axvline(b_mean, linestyle='--', alpha=0.5, color='b')              
        # Fill between for the two-tail areas with different alphas
        ax.fill_between(x, y, where=(x > b_conf[0]) & (x < b_conf[1]), alpha=0.1, label=f'{.95:.00%} CI [{b_conf[0]:.3f}, {b_conf[1]:.3f}]')
        # text
        ax.text(b_conf[1], max(y)/3, f'ub={b_conf[1]:.3f}', horizontalalignment='center', color='red', rotation=20, size=13)
        ax.text(b_conf[0], max(y)/5, f'lb={b_conf[0]:.3f}', horizontalalignment='center', color='green', rotation=20, size=13)
        ax.text(b_mean, 0, f'beta={b_mean:.3f}', horizontalalignment='center', color='b', rotation=0, size=13)
        # 
        # mark "t-statistic"
        ax.axvline(0, linestyle='--', alpha=0.5, color='k')
        ax.text(0, max(y)/7, f't-statistic\n={(b_mean/b_se):.3f}', horizontalalignment='center', color='k', rotation=20, size=13)
        ax.text(0, 0, f'p-value={(p_val):.3f}', horizontalalignment='center', color='k', rotation=20, size=13)
        # 
        ax.set_xlabel(f'{beta}')
        # ax.set_ylabel('Probability Density')
        # ax.set_title(f'{beta}')
        # ax.set_title('p.d.f. of X')
        ax.legend()
    #
    # plt.legend()
    plt.show()

# draw_ols_result(result_cps)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import patsy
import matplotlib.pyplot as plt


def ols_alt_spec(spec, data, show_res=True):
    y,X = patsy.dmatrices(spec, data=data, return_type='dataframe')
    model  = sm.OLS(y,X)
    result = model.fit(cov_type='HC0', use_t=True)
    if show_res : print(result.summary())
    return result

# Load the dataset
df_hps = pd.read_csv("https://raw.githubusercontent.com/SeanJSLee/Teaching_YU_DS_basic_KR/main/data/KOSIS_houshold_panel_survey/data_income_kor.csv")
df_hps.head(3)

## 연령그룹에 따라 추정해보기

In [ ]:
# 20대의 경우 
print('20대 OLS')
# ols_alt_spec(spec='ln_income ~ edu_year', data= df_hps.loc[df_hps['age'].isin(range(20,30))])
draw_ols_result(
    ols_alt_spec(spec='ln_income ~ edu_year', data= df_hps.loc[df_hps['age'].isin(range(20,30))])
    )

In [ ]:
# 30대의 경우 
print('30대 OLS')
ols_alt_spec(spec='ln_income ~ edu_year', data= df_hps.loc[df_hps['age'].isin(range(30,40))])

In [ ]:
# 40대의 경우 
print('40대 OLS')
ols_alt_spec(spec='ln_income ~ edu_year', data= df_hps.loc[df_hps['age'].isin(range(40,50))])

In [ ]:
# 50대의 경우 
print('50대 OLS')
ols_alt_spec(spec='ln_income ~ edu_year', data= df_hps.loc[df_hps['age'].isin(range(50,60))])

### Alternative specification
- 임금과 연관된 다른 변수 추가해보기 - 경력을 근사 할 수 있는 나이

In [ ]:
result_spec = {}
result_spec['original'] = ols_alt_spec(spec='ln_income ~ edu_year', data= df_hps)
draw_ols_result(result_spec['original'])
print('\n\n\n\n')
result_spec['alt_age'] = ols_alt_spec(spec='ln_income ~ edu_year + age', data= df_hps)
draw_ols_result(result_spec['alt_age'])
print('\n\n\n\n')
result_spec['alt_age_sq'] = ols_alt_spec(spec='ln_income ~ edu_year + age + np.power(age,2)', data= df_hps)
draw_ols_result(result_spec['alt_age_sq'])

In [ ]:
for model in result_spec.keys() :
    print(model, 'RMSE=', round(result_spec[model].mse_resid ** .5, 4))

In [ ]:


# fitted value df
df_predict = pd.DataFrame(columns=result_spec['alt_age_sq'].params.index)
df_predict['edu_year'] = range(0,22)
df_predict['Intercept'] = 1
df_predict['age'] = df_hps['age'].mean()
df_predict['np.power(age, 2)'] = df_hps['age'].mean() ** 2
df_predict


In [ ]:
fig, ax = plt.subplots(figsize=(8,8), sharex = True)


# actual values
ax.scatter(df_hps['edu_year'], df_hps['ln_income'], alpha=0.1, s=5, marker='D', c='b')


# 교육연도별 임금평균 - alternative specification
CEF_income_edu = df_hps.groupby('edu_year')['ln_income'].mean()
# 
ax.plot(CEF_income_edu, linestyle='-', marker='o', c='b')


for model in result_spec.keys() :
    ax.plot(result_spec[model].predict(df_predict[result_spec[model].params.index]))

# HS
edu_year = 12
ax.axvline(edu_year, color='0.5', linestyle='--', label='HS')
# Post-secondary
edu_year = 16
ax.axvline(edu_year, color='0.5', linestyle='--', label='PS')
    
plt.legend(['Actual', 'CEF', 'Fitted', '+age', '+age+$age^2$'])
plt.show()